In [1]:
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import time

In [2]:

batch_size = 128


In [3]:
#usr cifar 10 dataset and extract
cifar10.maybe_download_and_extract()

In [4]:
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'
image_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size = batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [5]:
image_test, labels_test = cifar10_input.inputs(eval_data=True, data_dir = data_dir, batch_size = batch_size)
print image_test.shape

(128, 24, 24, 3)


In [6]:
#create inputs's placeholder， 特征和lable，batch_size在之后定义网络结构时用到了，所以数据尺寸中的第一个值即样本条数需要被预先设定
image_holder = tf.placeholder(tf.float32,  [batch_size, 24, 24,3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [7]:
def variable_with_weight_loss(shape, stddev, wl):
    var =tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl,name = 'weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [8]:
#the first network
weight1 = variable_with_weight_loss(shape=[5,5,3,64], stddev=5e-2,  wl=0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1,1,1,1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4,bias=1.0, alpha=0.001/9.0, beta=0.75)

In [9]:
#the sercond network
weight2 = variable_with_weight_loss(shape=[5,5,64,64], stddev=5e-2, wl=0.00)
kernel2 = tf.nn.conv2d(norm1, weight2, [1,1,1,1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4,bias=1.0, alpha=0.001/9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

In [10]:
#after the conv layer, full connect layer
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3)+bias3)

In [11]:
#this is full connect layer, too
weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl = 0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

In [12]:
#the last layer
weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192.0, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logis = tf.add(tf.matmul(local4, weight5), bias5)

In [13]:
#loss
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


In [14]:
loss = loss(logis, label_holder)

In [15]:
#优化器选择AdamOptimizer
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [16]:
#get top k result
top_k_op = tf.nn.in_top_k(logis, label_holder, 1)

In [17]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [18]:
tf.train.start_queue_runners()

[<Thread(Thread-4, started daemon 139810391979776)>,
 <Thread(Thread-5, started daemon 139810043852544)>,
 <Thread(Thread-6, started daemon 139810035459840)>,
 <Thread(Thread-7, started daemon 139810027067136)>,
 <Thread(Thread-8, started daemon 139810018674432)>,
 <Thread(Thread-9, started daemon 139810010281728)>,
 <Thread(Thread-10, started daemon 139810001889024)>,
 <Thread(Thread-11, started daemon 139809993496320)>,
 <Thread(Thread-12, started daemon 139809775417088)>,
 <Thread(Thread-13, started daemon 139809767024384)>,
 <Thread(Thread-14, started daemon 139809758631680)>,
 <Thread(Thread-15, started daemon 139809750238976)>,
 <Thread(Thread-16, started daemon 139809741846272)>,
 <Thread(Thread-17, started daemon 139809733453568)>,
 <Thread(Thread-18, started daemon 139809725060864)>,
 <Thread(Thread-19, started daemon 139809641199360)>,
 <Thread(Thread-20, started daemon 139809632806656)>,
 <Thread(Thread-21, started daemon 139809624413952)>,
 <Thread(Thread-22, started daemon

In [19]:
#begin to train
max_step = 4000
for step in range(max_step):
    start_time = time.time()
    image_batch, label_batch = sess.run([image_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict={image_holder:image_batch, label_holder:label_batch})
    duration = time.time() - start_time
    if step%50==0:
        examples_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        format_str = ('step %d, loss=%.2f (%.1f example/sec; %.3f sec/per_batch)')
        print(format_str %(step, loss_value, examples_per_sec, sec_per_batch))

step 0, loss=4.68 (16.2 example/sec; 7.912 sec/per_batch)
step 50, loss=2.22 (127.8 example/sec; 1.002 sec/per_batch)
step 100, loss=2.00 (131.3 example/sec; 0.975 sec/per_batch)
step 150, loss=1.68 (127.1 example/sec; 1.007 sec/per_batch)
step 200, loss=1.69 (128.9 example/sec; 0.993 sec/per_batch)
step 250, loss=1.53 (127.7 example/sec; 1.002 sec/per_batch)
step 300, loss=1.93 (70.4 example/sec; 1.819 sec/per_batch)
step 350, loss=1.47 (126.8 example/sec; 1.009 sec/per_batch)
step 400, loss=1.53 (126.6 example/sec; 1.011 sec/per_batch)
step 450, loss=1.36 (81.0 example/sec; 1.580 sec/per_batch)
step 500, loss=1.49 (122.6 example/sec; 1.044 sec/per_batch)
step 550, loss=1.39 (91.2 example/sec; 1.403 sec/per_batch)
step 600, loss=1.35 (128.7 example/sec; 0.994 sec/per_batch)
step 650, loss=1.64 (116.5 example/sec; 1.099 sec/per_batch)
step 700, loss=1.46 (116.2 example/sec; 1.102 sec/per_batch)
step 750, loss=1.31 (76.5 example/sec; 1.673 sec/per_batch)
step 800, loss=1.19 (118.9 examp

In [20]:
saver = tf.train.Saver()
model_dir = "/data/1xiu/models/cifar"
model_name = "cifar_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
saver.save(sess,os.path.join(model_dir, model_name))

saver.restore(sess, os.path.join(model_dir, model_name))

INFO:tensorflow:Restoring parameters from /data/1xiu/models/cifar/cifar_model


In [21]:
num_examples = 10000
import math


In [22]:
num_iter = int(math.ceil(num_examples/batch_size))
print(num_examples, batch_size, num_iter)
true_count=0
total_sample_count = num_iter*batch_size
step = 0
while step<num_iter:
    image_batch, label_batch = sess.run([image_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder:image_batch, label_holder:label_batch})
    true_count += np.sum(predictions)
    if step %10==0:
        print true_count
    step += 1

(10000, 128, 78)
94
1036
1965
2892
3856
4808
5725
6650


In [23]:
precision = true_count*1.0 / total_sample_count
print('precision @ =%.3f' %precision)

precision @ =0.733
